In [1]:
import pandas as pd
import nltk
import numpy as np
from functools import reduce
import math


In [2]:
NOME_DO_ARQUIVO = 'estadao_noticias_eleicao.csv'
indice_invertido = {}
dados = pd.read_csv(NOME_DO_ARQUIVO)
dados = dados.replace(np.nan, '', regex=True)
dados["TF"] = None

In [24]:
def geraIndice():
    for i, row in dados.iterrows():
        
        titulo = (word.lower() for word in (nltk.word_tokenize(row['titulo'])))
        subtitulo = (word.lower() for word in (nltk.word_tokenize(row['subTitulo'])))
        conteudo = (word.lower() for word in (nltk.word_tokenize(row['conteudo'])))
        palavras_divididas = list(titulo) + list(subtitulo) + list(conteudo)
        row["TF"] = {}
        for palavra in palavras_divididas:
            if palavra not in indice_invertido:
                indice_invertido[palavra] = dict([row['idNoticia']])
            else:
                indice_invertido[palavra].add(row['idNoticia'])
            if palavra not in row["TF"]:
                row["TF"][palavra] = 0
            row["TF"][palavra] += 1

In [4]:
def modeloBinario(termos):
    return list(reduce((lambda x,y: indice_invertido[x] & indice_invertido[y]), termos))

In [5]:
def getDocumentsWithTerms(terms):
    return list(reduce((lambda x,y: indice_invertido[x].union(indice_invertido[y])), terms))

In [6]:
def generateQueryTF(query):
    tf = {}
    for term in query:
        if term not in tf:
            tf[term] = 0
        tf[term] += 1
    return tf

In [7]:
def modeloTF(termos):
    documentos = modeloBinario(termos)
    documentsTF = []
    for idDoc in documentos:
        documentWeight = 0
        for termo in termos:
            docTF = dados.TF[idDoc][termo] if (termo in dados.TF[idDoc]) else 0
            qTF = termos.count(termo)
            documentWeight += docTF * qTF
            
        documentsTF.append((idDoc, documentWeight))
    documentsTF.sort(key=lambda tup: tup[1], reverse=True)
    ids = []
    for tupla in documentsTF:
        ids.append(tupla[0])
    return ids

In [45]:
def modeloIDF(termos):
    documentos = modeloBinario(termos)
    documentsTF = []
    for idDoc in documentos:
        documentWeight = 0
        for termo in termos:
            docTF = dados.TF[idDoc][termo] if (termo in dados.TF[idDoc]) else 0
            qTF = termos.count(termo)
            idf = math.log(dados.titulo.size/len(indice_invertido[termo]))
            documentWeight += docTF * qTF * idf
            
        documentsTF.append((idDoc, documentWeight))
    documentsTF.sort(key=lambda tup: tup[1], reverse=True)
    ids = []
    for tupla in documentsTF:
        ids.append(tupla[0])
    return ids

In [46]:
# geraIndice()
termos = ["segundo", "turno"]
consultaTF = modeloTF(termos)[:5]
consultaIDF = modeloIDF(termos)[:5]

[2744, 7, 2112, 7672, 2388]
[2744, 2112, 7672, 1235, 2388]

print(consultaTF)
print(consultaIDF)

[2111, 2177, 2137, 2113, 2293]
[2111, 2177, 1963, 2137, 2113]


In [47]:
oi = modeloBinario(["pt", "espera", "30", "mil"])

TypeError: unhashable type: 'set'